##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to modules, layers, and models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/intro_to_modules"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_modules.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/intro_to_modules.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/intro_to_modules.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

To do machine learning in TensorFlow, you are likely to need to define, save, and restore a model.

A model is, abstractly: 

* A function that computes something on tensors (a **forward pass**)
* Some variables that can be updated in response to training

In this guide, you will go below the surface of Keras to see how TensorFlow models are defined. This looks at how TensorFlow collects variables and models, as well as how they are saved and restored.

Note: If you instead want to immediately get started with Keras, please see [the collection of Keras guides](./keras/).


TensorFlow에서 기계 학습을 수행하려면 `Model`을 정의, 저장 및 복원해야 할 수 있습니다.

모델은 추상적으로 다음과 같습니다.

* tensor에서 무언가를 계산하는 함수(forward pass)
* 훈련에 대한 응답으로 업데이트될 수 있는 일부 변수

이 가이드에서는 TensorFlow 모델이 어떻게 정의되는지 알아보기 위해 Keras 표면 아래로 이동합니다. 여기서는 TensorFlow가 변수와 모델을 수집하는 방법과 저장 및 복원 방법을 살펴봅니다.

참고: 대신 Keras를 즉시 시작하려면 Keras 가이드 모음을 참조하세요.

## Setup

In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

## Defining models and layers in TensorFlow

Most models are made of layers. Layers are functions with a known mathematical structure that can be reused and have trainable variables.  In TensorFlow, most high-level implementations of layers and models, such as Keras or [Sonnet](https://github.com/deepmind/sonnet), are built on the same foundational class: `tf.Module`.

Here's an example of a very simple `tf.Module` that operates on a scalar tensor:


대부분의 모델은 layer들로 구성됩니다. layer들은 재사용할 수 있고 학습 가능한 변수가 있는 알려진 수학적 구조를 가진 함수입니다. TensorFlow에서 Keras 또는 Sonnet과 같은 계층 및 모델에 대한 대부분의 고수준 구현은 동일한 기본 클래스인 tf.Module로 만들어집니다.

다음은 스칼라 텐서에서 작동하는 매우 간단한 `tf.Module`의 예입니다.

In [2]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

In [3]:
simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0)) # called __call__

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

Modules and, by extension, layers are deep-learning terminology for "objects": they have internal state, and methods that use that state.

There is nothing special about `__call__` except to act like a [Python callable](https://stackoverflow.com/questions/111234/what-is-a-callable); you can invoke your models with whatever functions you wish.

You can set the trainability of variables on and off for any reason, including freezing layers and variables during fine-tuning.

Note: **`tf.Module` is the base class for both `tf.keras.layers.Layer` and `tf.keras.Model`, so everything you come across here also applies in Keras.**  For historical compatibility reasons Keras layers do not collect variables from modules, so your models should use only modules or only Keras layers.  However, the methods shown below for inspecting variables are the same in either case.

By subclassing `tf.Module`, any `tf.Variable` or `tf.Module` instances assigned to this object's properties are automatically collected.  This allows you to save and load variables, and also create collections of `tf.Module`s.

모듈과 레이어는 "객체"에 대한 딥 러닝 용어입니다. <b>객체는 내부 state와 이러한 state를 사용하는 메서드가 있습니다.</b>

Python callable처럼 작동하는 것을 제외하고 `__call__`에 대해 특별한 것은 없습니다. 원하는 함수들로 여러분의 모델을 호출할 수 있습니다.

fine-tuning 중 layer 및 변수 freezing을 포함하여 어떤 이유로든 변수의 trainability을 켜고 끌 수 있습니다.

참고: tf.Module은 tf.keras.layers.Layer 및 tf.keras.Model의 기본 클래스이므로 여기에서 보는 모든 것이 Keras에도 적용됩니다. <font color="blue"><b>역사적 호환성을 위해 Keras 계층은 module에서 tf.Variables를 수집하지 않으므로 모델은 module만 사용하거나 Keras layer들만 사용해야 합니다. 그러나 아래에 표시된 변수 검사 방법은 두 경우 모두 동일합니다.<b></font>

tf.Module을 서브클래싱하면 이 객체의 속성에 할당된 모든 tf.Variable 또는 tf.Module 인스턴스가 자동으로 수집됩니다. 이를 통해 변수를 저장 및 로드하고 tf.Modules 컬렉션을 만들 수도 있습니다.

In [4]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables) # trainable_variables는 tf.Module의 속성임. = trainable_variables = a_variable
print("*" * 60)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
************************************************************
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


This is an example of a two-layer linear layer model made out of modules.

First a dense (linear) layer:

이것은 모듈로 구성된 two-layer linear 모델의 예입니다.

먼저 dense(linear) layer:

In [ ]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    # print("self.w=", self.w)
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

And then the complete model, which makes two layer instances and applies them:

In [ ]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

In [ ]:
# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.        2.4882534]], shape=(1, 2), dtype=float32)


`tf.Module` instances will automatically collect, recursively, any `tf.Variable` or `tf.Module` instances assigned to it. This allows you to manage collections of `tf.Module`s with a single model instance, and save and load whole models.

tf.Module 인스턴스는 할당된 모든 tf.Variable 또는 tf.Module 인스턴스를 재귀적으로 자동으로 수집합니다. 이를 통해 단일 모델 인스턴스로 tf.Modules 컬렉션을 관리하고 전체 모델을 저장하고 로드할 수 있습니다.

In [ ]:
print("Submodules:", my_model.submodules)


Submodules: (<__main__.Dense object at 0x7fde03284150>, <__main__.Dense object at 0x7fde03284490>)


In [ ]:
for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-1.3337831 , -0.96477866,  0.95781565],
       [ 0.94454396, -1.0532169 ,  0.7352832 ],
       [-0.5247758 , -0.7555854 , -0.09246594]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 2.4640367 ,  0.27036673],
       [-0.1620068 , -0.81206304],
       [-1.2079171 ,  0.7772717 ]], dtype=float32)> 



### Waiting to create variables

You may have noticed here that you have to define both input and output sizes to the layer.  This is so the `w` variable has a known shape and can be allocated.

By deferring variable creation to the first time the module is called with a specific input shape, you do not need specify the input size up front.

여기서 레이어에 대한 입력 및 출력 크기를 모두 정의해야 한다는 것을 눈치채셨을 것입니다. 이것은 w 변수가 알려진 모양을 갖고 할당될 수 있도록 하기 위한 것입니다.

모듈이 특정 입력 형태로 처음 호출될 때 변수 생성을 연기하면 입력 크기를 미리 지정할 필요가 없습니다.

In [ ]:
class FlexibleDenseModule(tf.Module):
  # Note: No need for `in_features`
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    print("x.shape=", x.shape)
    print("x.shape[-1]=", x.shape[-1])
    # Create variables on first call.
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [ ]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)



In [ ]:
my_model = MySequentialModule(name="the_model")

In [ ]:
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

x.shape= (1, 3)
x.shape[-1]= 3
x.shape= (1, 3)
x.shape[-1]= 3
Model results: tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


This flexibility is why TensorFlow layers often only need to specify the shape of their outputs, such as in `tf.keras.layers.Dense`, rather than both  the input and output size.

이러한 유연성 때문에 TensorFlow layer는 종종 입력 및 출력 크기가 아닌 tf.keras.layers.Dense와 같이 출력의 모양만 지정하면 됩니다.

## Saving weights

You can save a `tf.Module` as both a [checkpoint](https://www.tensorflow.org/guide/checkpoint) and a [SavedModel](https://www.tensorflow.org/guide/saved_model).

Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules):

tf.Module을 체크포인트와 저장된 모델로 저장할 수 있습니다.

체크포인트는 단지 weight(즉, 모듈과 그 하위 모듈 내부의 변수 집합의 값)입니다.

In [ ]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

Checkpoints consist of two kinds of files: the data itself and an index file for metadata. The index file keeps track of what is actually saved and the numbering of checkpoints, while the checkpoint data contains the variable values and their attribute lookup paths.

체크포인트는 데이터 자체와 메타데이터용 인덱스 파일의 두 가지 파일로 구성됩니다. 인덱스 파일은 실제로 저장된 항목과 체크포인트의 번호를 추적하는 반면 체크포인트 데이터에는 변수 값과 속성 조회 경로가 포함됩니다.

In [ ]:
!ls my_checkpoint*

my_checkpoint.data-00000-of-00001  my_checkpoint.index


You can look inside a checkpoint to be sure the whole collection of variables is saved, sorted by the Python object that contains them.

checkpoint 내부를 살펴보고 variables의 전체 컬렉션이 저장되고 variables가 포함된 Python 개체에 따라 정렬되었는지 확인할 수 있습니다.

In [ ]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

During distributed (multi-machine) training they can be sharded,  which is why they are numbered (e.g., '00000-of-00001').  In this case, though, there is only have one shard.

When you load models back in, you overwrite the values in your Python object.

분산(다중 머신) 학습 중에 샤딩될 수 있으므로 번호가 매겨집니다(예: '00000-of-00001'). 그러나 이 경우에는 하나의 샤드만 있습니다.

모델을 다시 로드할 때 Python 객체의 값을 덮어씁니다.

In [ ]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

In [ ]:
# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

x.shape= (1, 3)
x.shape[-1]= 3
x.shape= (1, 3)
x.shape[-1]= 3


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>

Note: As checkpoints are at the heart of long training workflows `tf.checkpoint.CheckpointManager` is a helper class that makes checkpoint management much easier. Refer to the [Training checkpoints guide](https://www.tensorflow.org/guide/checkpoint) for more details.

참고: 체크포인트는 긴 교육 워크플로의 핵심이므로 tf.checkpoint.CheckpointManager는 체크포인트 관리를 훨씬 쉽게 만드는 도우미 클래스입니다. 자세한 내용은 Training checkpoints guide 를 참조하세요.

## Saving `Function`s

TensorFlow can run models without the original Python objects, as demonstrated by [TensorFlow Serving](https://tensorflow.org/tfx) and [TensorFlow Lite](https://tensorflow.org/lite), even when you download a trained model from [TensorFlow Hub](https://tensorflow.org/hub).

TensorFlow needs to know how to do the computations described in Python, but **without the original code**. To do this, you can make a **graph**, which is described in the [Introduction to graphs and functions guide](./intro_to_graphs.ipynb).

This graph contains operations, or *ops*, that implement the function.

You can define a graph in the model above by adding the `@tf.function` decorator to indicate that this code should run as a graph.

TensorFlow Hub에서 사전 훈련된(Pre-Trained) 모델을 다운로드하는 경우에도 TensorFlow Serving 및 TensorFlow Lite에서 설명한 것처럼 TensorFlow는 원본 Python 객체 없이 모델을 실행할 수 있습니다.

TensorFlow는 원본 코드 없이 Python으로 작성된 계산을 수행하는 방법을 알아야 합니다. 이를 위해 그래프 및 함수 소개 가이드에 설명된 그래프를 만들 수 있습니다.

이 그래프에는 함수를 구현하는 작업 또는 작업이 포함되어 있습니다.

이 코드가 그래프로 실행되어야 함을 나타내기 위해 @tf.function 데코레이터를 추가하여 위의 모델에서 그래프를 정의할 수 있습니다.

In [ ]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    print("MySequentialModule callable")
    x = self.dense_1(x)
    return self.dense_2(x)


In [ ]:
####################################################
# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")
####################################################

The module you have made works exactly the same as before.  Each unique signature passed into the function creates a separate graph. Check the [Introduction to graphs and functions guide](./intro_to_graphs.ipynb) for details.

당신이 만든 모듈은 이전과 정확히 동일하게 작동합니다. 함수에 전달된 각각의 고유한 signature은 별도의 그래프를 생성합니다. 자세한 내용은 그래프 및 함수 소개 가이드를 확인하세요.

In [ ]:
print("my_model([[2.0, 2.0, 2.0]])=", my_model([[2.0, 2.0, 2.0]]), "\n")
print("my_model([[3.0, 3.0, 3.0]])=", my_model([[3.0, 3.0, 3.0]]), "\n")
print("my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]])", my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

MySequentialModule callable
my_model([[2.0, 2.0, 2.0]])= tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32) 

MySequentialModule callable
my_model([[3.0, 3.0, 3.0]])= tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32) 

MySequentialModule callable
my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]) tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


In [ ]:
# print concrete_signatures!!!

You can visualize the graph by tracing it within a TensorBoard summary.

In [ ]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)

tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

MySequentialModule callable
tf.Tensor([[0.        1.7008992]], shape=(1, 2), dtype=float32)


Launch TensorBoard to view the resulting trace:

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir logs/func

### Creating a `SavedModel`

The recommended way of sharing completely trained models is to use `SavedModel`.  `SavedModel` contains both a collection of functions and a collection of weights. 

You can save the model you have just trained as follows:

완전히 훈련된 모델을 공유하는 권장 방법은 SavedModel을 사용하는 것입니다. SavedModel에는 함수 컬렉션과 weight 컬렉션이 모두 포함되어 있습니다.

방금 훈련한 모델을 다음과 같이 저장할 수 있습니다.

In [ ]:
tf.saved_model.save(my_model, "the_saved_model")

MySequentialModule callable
MySequentialModule callable
MySequentialModule callable


In [ ]:
# Inspect the SavedModel in the directory
!ls -l the_saved_model

total 24
drwxr-xr-x 2 root root  4096 Oct  7 01:49 assets
-rw-r--r-- 1 root root 15989 Oct  7 01:49 saved_model.pb
drwxr-xr-x 2 root root  4096 Oct  7 01:49 variables


In [ ]:
# The variables/ directory contains a checkpoint of the variables 
!ls -l the_saved_model/variables

total 8
-rw-r--r-- 1 root root 456 Oct  7 01:49 variables.data-00000-of-00001
-rw-r--r-- 1 root root 356 Oct  7 01:49 variables.index


The `saved_model.pb` file is a [protocol buffer](https://developers.google.com/protocol-buffers) describing the functional `tf.Graph`.

Models and layers can be loaded from this representation without actually making an instance of the class that created it.  This is desired in situations where you do not have (or want) a Python interpreter, such as serving at scale or on an edge device, or in situations where the original Python code is not available or practical to use.

You can load the model as new object:

<font color="red">saved_model.pb 파일은 functional tf.Graph를 설명하는 프로토콜 버퍼입니다.</font>

모델과 레이어는 실제로 모델을 생성시킨 클래스의 인스턴스를 만들지 않고도 이 represent에서 로드될 수 있습니다. 이는 대규모 또는 에지 장치에서 제공하는 것과 같이 Python 인터프리터가 없거나 원하지 않는 상황이나 원래 Python 코드를 사용할 수 없거나 사용할 수 없는 상황에서 바람직합니다.

모델을 새로운 객체로 로드할 수 있습니다.

In [ ]:
new_model = tf.saved_model.load("the_saved_model")

`new_model`, created from loading a saved model, is an internal TensorFlow user object without any of the class knowledge. It is not of type `SequentialModule`.

<font color="red">**저장된 모델을 로드하여 생성된 `new_model`은 클래스 지식이 없는 내부 TensorFlow 유저 객체입니다. SequentialModule 유형이 아닙니다.**</font>

In [ ]:
isinstance(new_model, SequentialModule)

False

This new model works on the already-defined input signatures. You can't add more signatures to a model restored like this.

이 새 모델은 이미 정의된 입력 signatures에서 작동합니다. 이렇게 복원된 모델에는 signatures을 더 추가할 수 없습니다.

In [ ]:
print(new_model([[2.0, 2.0, 2.0]]))
print(new_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


In [ ]:
print(new_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

ValueError: ignored

Thus, using `SavedModel`, you are able to save TensorFlow weights and graphs using `tf.Module`, and then load them again.

따라서 SavedModel을 사용하면 tf.Module을 사용하여 TensorFlow weight와 그래프를 저장하고 다시 로드할 수 있습니다.

## Keras models and layers

Note that up until this point, there is no mention of Keras. You can build your own high-level API on top of `tf.Module`, and people have.  

In this section, you will examine how Keras uses `tf.Module`.  A complete user guide to Keras models can be found in the [Keras guide](https://www.tensorflow.org/guide/keras/sequential_model).


여기까지는 Keras에 대한 언급이 없습니다. tf.Module 위에 여러분만의 고수준 API를 구축할 수 있습니다

이 섹션에서는 Keras가 tf.Module을 사용하는 방법을 살펴봅니다. Keras 모델에 대한 전체 사용자 가이드는 Keras 가이드에서 찾을 수 있습니다.

### Keras layers

`tf.keras.layers.Layer` is the base class of all Keras layers, and it inherits from `tf.Module`.

You can convert a module into a Keras layer just by swapping out the parent and then changing `__call__` to `call`:

tf.keras.layers.Layer는 모든 Keras 계층의 기본 클래스이며 tf.Module에서 상속됩니다.

`tf.Module` 상속을 `tf.keras.layers.Layer` 상속으로 교체한 다음 기존 `__call__` 함수를 `call` 함수로 변경하여 tf.Module을 Keras Layer로 변환할 수 있습니다.

In [ ]:
class MyDense(tf.keras.layers.Layer):
  # Adding **kwargs to support base Keras layer arguments
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # This will soon move to the build step; see below
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [ ]:
simple_layer = MyDense(name="simple", in_features=3, out_features=3)

Keras layers have their own `__call__` that does some bookkeeping described in the next section and then calls `call()`. You should notice no change in functionality.

Keras 레이어에는 다음 섹션에서 설명하는 일부 bookkeeping를 수행한 다음 call()을 호출하는 자체 __call__ 함수가 있습니다. 기능에 변화가 없음을 알 수 있습니다.

In [ ]:
simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.67874813, 4.032743  , 3.017374  ]], dtype=float32)>

### The `build` step

As noted, it's convenient in many cases to wait to create variables until you are sure of the input shape.

Keras layers come with an extra lifecycle step that allows you more flexibility in how you define your layers. This is defined in the `build` function.

`build` is called exactly once, and it is called with the shape of the input. It's usually used to create variables (weights).

You can rewrite `MyDense` layer above to be flexible to the size of its inputs:


언급했듯이 입력 형태가 확실할 때까지 변수 생성을 기다리는 것이 많은 경우에 편리합니다.

Keras 레이어에는 레이어를 정의하는 방법에 더 많은 유연성을 제공하는 추가 lifecycle 단계가 있습니다. 이것은 빌드 함수에서 정의됩니다.

build 는 정확히 한 번 호출되며 입력의 shape으로 호출됩니다. 일반적으로 변수(weigth)를 만드는 데 사용됩니다.

위의 MyDense 레이어를 다시 작성하여 입력 크기에 유연하게 맞출 수 있습니다.

In [ ]:
class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    print("called FlexibleDense::build")
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    print("called FlexibleDense::call")
    return tf.matmul(inputs, self.w) + self.b



In [ ]:
# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

At this point, the model has not been built, so there are no variables:

이 시점에서 모델이 빌드되지 않았으므로 변수가 없습니다.

In [ ]:
flexible_dense.variables

[]

Calling the function allocates appropriately-sized variables:

함수[call()]를 호출하면 적절한 크기의 변수가 할당됩니다.

In [ ]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

called FlexibleDense::build
called FlexibleDense::call
Model results: tf.Tensor(
[[-0.778502   3.9722497 -1.746544 ]
 [-1.167753   5.9583745 -2.6198156]], shape=(2, 3), dtype=float32)


In [ ]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 1.0096889 ,  0.5873897 ,  0.0836022 ],
        [-0.7073977 ,  1.4682889 , -1.4798621 ],
        [-0.69154215, -0.06955379,  0.5229879 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

Since `build` is only called once, inputs will be rejected if the input shape is not compatible with the layer's variables:

`build`는 한 번만 호출되므로 input shape이 레이어의 variables와 호환되지 않으면 입력이 거부됩니다.

In [ ]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

called FlexibleDense::call
Failed: Exception encountered when calling layer "flexible_dense" (type FlexibleDense).

Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


Keras layers have a lot more extra features including:

* Optional losses
* Support for metrics
* Built-in support for an optional `training` argument to differentiate between training and inference use
* `get_config` and `from_config` methods that allow you to accurately store configurations to allow model cloning in Python

Read about them in the [full guide](./keras/custom_layers_and_models.ipynb) to custom layers and models.

Keras 레이어에는 다음과 같은 훨씬 더 많은 추가 기능이 있습니다.

* 선택할 수 있는 여러 loss 지원
* metric 지원
* training과 inference 사용을 구별하기 위한 선택적 training argument에 대한 내장 지원
* Python에서 모델 복제를 허용하도록 configuration을 정확하게 저장할 수 있는 get_config 및 from_config 메서드

사용자 지정 계층 및 모델에 대한 전체 가이드에서 이에 대해 읽어보세요.

### Keras models

You can define your model as nested Keras layers.

However, Keras also provides a full-featured model class called `tf.keras.Model`. It inherits from `tf.keras.layers.Layer`, so a Keras model can be used, nested, and saved in the same way as Keras layers. Keras models come with extra functionality that makes them easy to train, evaluate, load, save, and even train on multiple machines.

You can define the `SequentialModule` from above with nearly identical code, again converting `__call__` to `call()` and changing the parent:

모델을 중첩된 Keras 레이어로 정의할 수 있습니다.

그러나 Keras는 tf.keras.Model이라는 완전한 기능을 갖춘 모델 클래스도 제공합니다. tf.keras.layers.Layer에서 상속하므로 Keras.Model을 Keras.Layer와 동일한 방식으로 사용, 중첩 및 저장할 수 있습니다. Keras.Model에는 여러 머신에서 쉽게 훈련, 평가, 로드, 저장 및 훈련할 수 있는 추가 기능이 있습니다.

위와 거의 동일한 코드로 SequentialModule을 정의할 수 있습니다. 다시 `__call__`을 `call()`으로 변환하고 부모를 변경합니다.

In [ ]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

In [ ]:
# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

called FlexibleDense::build
called FlexibleDense::call
called FlexibleDense::build
called FlexibleDense::call
Model results: tf.Tensor([[3.6755495 3.989869 ]], shape=(1, 2), dtype=float32)


All the same features are available, including tracking variables and submodules.

Note: To emphasize the note above, a raw `tf.Module` nested inside a Keras layer or model will not get its variables collected for training or saving.  Instead, nest Keras layers inside of Keras layers.

tracking 변수 및 하위 모듈을 포함하여 모든 동일한 기능을 사용할 수 있습니다.

참고: 위의 참고 사항을 강조하기 위해 Keras.layer 또는 keras.modle 내부에 중첩된 raw tf.Module은 학습 또는 저장을 위해 수집된 변수를 가져오지 않습니다. 대신 Keras 레이어 내부에 Keras 레이어를 중첩합니다.

In [ ]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.55456114, -0.08233272,  0.44348752],
        [ 0.16435763, -1.0189123 ,  0.12058347],
        [-0.59768844,  1.1908588 ,  0.60987145]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[2.0176234 , 0.80732167],
        [0.320094  , 0.17715621],
        [1.3326826 , 1.6024525 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [ ]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x7fde00e40d90>,
 <__main__.FlexibleDense at 0x7fde00e403d0>)

Overriding `tf.keras.Model` is a very Pythonic approach to building TensorFlow models.  If you are migrating models from other frameworks, this can be very straightforward.

If you are constructing models that are simple assemblages of existing layers and inputs, you can save time and space by using the [functional API](./keras/functional.ipynb), which comes with additional features around model reconstruction and architecture.

Here is the same model with the functional API:

tf.keras.Model을 overriding하는 것은 TensorFlow 모델 구축에 대한 매우 Pythonic한 접근 방식입니다. 다른 프레임워크에서 모델을 마이그레이션하는 경우 매우 간단할 수 있습니다.

기존 레이어와 입력의 단순한 조합인 모델을 구성하는 경우 모델 재구성 및 아키텍처와 관련된 추가 기능과 함께 제공되는 `functional API`를 사용하여 시간과 공간을 절약할 수 있습니다.

다음은 `functional API`가 있는 동일한 모델입니다.

In [ ]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
print("x type=", type(x))
x = FlexibleDense(2)(x)

####
my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

called FlexibleDense::build
called FlexibleDense::call
x type= <class 'keras.engine.keras_tensor.KerasTensor'>
called FlexibleDense::build
called FlexibleDense::call
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 flexible_dense_5 (FlexibleD  (None, 3)                12        
 ense)                                                           
                                                                 
 flexible_dense_6 (FlexibleD  (None, 2)                8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [ ]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

called FlexibleDense::call
called FlexibleDense::call


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.7929738, 4.7242155]], dtype=float32)>

The major difference here is that the input shape is specified up front as part of the functional construction process. The `input_shape` argument in this case does not have to be completely specified; you can leave some dimensions as `None`.

Note: You do not need to specify `input_shape` or an `InputLayer` in a subclassed model; these arguments and layers will be ignored.

여기서 주요 차이점은 입력 shape이 기능 구성 프로세스의 일부로 미리 지정된다는 것입니다. 이 경우 input_shape 인수를 완전히 지정할 필요는 없습니다. 일부 demension는 None으로 둘 수 있습니다.

참고: 서브클래싱된 모델에서는 input_shape 또는 InputLayer를 지정할 필요가 없습니다. 이러한 인수와 레이어는 무시됩니다.

## Saving Keras models

Keras models can be checkpointed, and that will look the same as `tf.Module`.

Keras models can also be saved with `tf.saved_model.save()`, as they are modules.  However, Keras models have convenience methods and other functionality:

Keras 모델은 checkpoint을 지정할 수 있으며 이는 tf.Module과 동일하게 보입니다.

Keras 모델은 모듈이므로 tf.saved_model.save()로 저장할 수도 있습니다. 그러나 Keras 모델에는 편리한 방법 및 기타 기능이 있습니다.

In [ ]:
my_sequential_model.save("exname_of_file")

called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call
called FlexibleDense::call


Just as easily, they can be loaded back in:

In [ ]:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

Keras `SavedModels` also save metric, loss, and optimizer states.

This reconstructed model can be used and will produce the same result when called on the same data:

In [ ]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.6755495, 3.989869 ]], dtype=float32)>

There is more to know about saving and serialization of Keras models, including providing configuration methods for custom layers for feature support. Check out the [guide to saving and serialization](https://www.tensorflow.org/guide/keras/save_and_serialize).

# What's next

If you want to know more details about Keras, you can follow the existing Keras guides [here](./keras/).

Another example of a high-level API built on `tf.module` is Sonnet from DeepMind, which is covered on [their site](https://github.com/deepmind/sonnet).